# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/abdel-raouf/Data_Modeling_With_Cassandra_Project


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
         # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

# pickling in python
with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in our csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding portion of the project. 

## Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Creating a cluster and a keySpace.

#### Creating a Cluster

In [5]:
# making a connection to a Cassandra instance on our local machine. 
from cassandra.cluster import Cluster
cluster = Cluster()

# Establish connection and begin executing queries.
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace (our database)
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS music_app
        WITH REPLICATION =
        {'class' : 'SimpleStrategy', 'replication_factor' : 2}
    """)
    
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('music_app')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra we model the database tables on the queries we want to run.

## Creating queries to ask the following three questions of the data

### Query_1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### Query_2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### Query_3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query_1

In [8]:
# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
# sessionId = 338, and itemInSession = 4

# Creating Table.
query = "CREATE TABLE IF NOT EXISTS songs_heard "
query = query + "(artist_name TEXT, title TEXT, duration FLOAT, session_id INT, item_in_session INT, PRIMARY KEY (session_id, item_in_session))" 
try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
# loading data to the songs_heard table from the `event_datafile_new.csv` file (ETL).
file = 'event_datafile_new.csv'

# pickling in python.
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_heard (artist_name, title, duration, session_id, item_in_session)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        # we need to cast to the data-type we declared for the column in the table creation except for the TEXT data-type, as csvreader return a list of strings 
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

In [39]:
# Add in the SELECT statement to verify the data was entered into the table.
query = "SELECT * FROM songs_heard WHERE session_id=338 AND item_in_session=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

data = []    
for row in rows:
    data.append([row.artist_name, row.title, row.duration])
    
df = pd.DataFrame(data, columns = ['artist_name', 'title', 'duration'])
df

artist_name                            title    duration
0   Faithless  Music Matters (Mark Knight Dub)  495.307312

### Query_2

In [11]:
# TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
# for userid = 10, sessionid = 182

# Creating Table.
query = "CREATE TABLE IF NOT EXISTS users_history "
query = query + "(artist_name TEXT, song_title TEXT, first_name TEXT, last_name TEXT, user_id INT, session_id INT, item_in_session INT, PRIMARY KEY(user_id, session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)        

In [12]:
# loading data to the users_history table from the `event_datafile_new.csv` file (ETL).
file = 'event_datafile_new.csv'

# pickling in python
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_history (artist_name, song_title, first_name, last_name, user_id,  session_id, item_in_session)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        # we need to cast to the data-type we declared for the column in the table creation except for the TEXT data-type, as csvreader return a list of strings 
        session.execute(query, (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3])))

In [38]:
# Add in the SELECT statement to verify the data was entered into the table.
query = "SELECT * FROM users_history WHERE user_id=10 AND session_id=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

data= []    
for row in rows:
    data.append([row.artist_name, row.song_title, row.first_name, row.last_name])
    
df = pd.DataFrame(data, columns = ['artist_name', 'song_title', 'first_name', 'last_name'])
df

artist_name                                         song_title  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  first_name last_name  
0     Sylvie      Cruz  
1     Sylvie      Cruz  
2     Sylvie      Cruz  
3     Sylvie      Cruz

In [14]:
# TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# Creating Table.
query = "CREATE TABLE IF NOT EXISTS users_per_song "
query = query + "(first_name TEXT, last_name TEXT, song_title TEXT, user_id INT, PRIMARY KEY (song_title, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)               

In [16]:
# loading data to the user_per_song table from the `event_datafile_new.csv` file (ETL).
file = 'event_datafile_new.csv'

# pickling in python
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_per_song (first_name, last_name, song_title, user_id)"
        query = query + " VALUES (%s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        # we need to cast to the data-type we declared for the column in the table creation except for the TEXT data-type, as csvreader return a list of strings 
        session.execute(query, (line[1], line[4], line[9], int(line[10])))

In [37]:
# Add in the SELECT statement to verify the data was entered into the table
query = "SELECT * FROM users_per_song WHERE song_title='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

data = []
for row in rows:
    data.append([row.first_name, row.last_name])
    
df = pd.DataFrame(data, columns = ['first_name', 'last_name'])
df

first_name last_name
0  Jacqueline     Lynch
1       Tegan    Levine
2        Sara   Johnson

### Drop the tables before closing out the sessions

In [20]:
# Drop tables before closing out the session.
query = "DROP TABLE IF EXISTS songs_heard"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS users_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS user_per_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()